# An Introduction to GeoFence Model
_**GeoFence model helps determines workers who are at a work site, if they are in high consequence areas, how long employees or equipment were within the geo fenced work site, and what activities or movements occurred.**_

1. [Introduction](#1.-Introduction)
2. [Preparing JSON](#2.-Preparing-JSON)
3. [Method - 1 Using Endpoint](#3.-Using-Endpoint)<br>
    3.1. [Creating model](#3.1.-Creating-Model)<br>
    3.2. [Creating endpoint config](#3.2.-Creating-Endpoint-Config)<br>
    3.3. [Creating endpoint](#3.3.-Creating-Endpoint)<br>
    3.2. [Invoking endpoint](#3.4.-Invoking-Endpoint)<br>
    3.2. [Deleting the Endpoint - Optional](#3.5.-Deleting-the-Endpoint---Optional)
4. [Method - 2 Using batch transform job](#4.-Using-Batch-Transform-Job)
5. [Model Output](#5.-Model-Output)


## 1. Introduction
Geo Fencing Model informs an organization of employee and equipment location and the activities and movements within that location.Jump start your ability to understand worksite risk using the Modjoul Geo Fence Model. Geo fencing uses GPS to establish a digital boundary around a work site to keep employees safe. Understand which workers enter into a work site and understand when employees are in high consequence areas. The Geo Fence Model provides output in JSON Format.Useful outputs of duration and counts of employee and equipment within the geo fenced location. Geofences can be Circles or Polygons.

## 2. Preparing JSON
Prepare a Json file that should and must contain input **latitude and longitude** values and at least one geofence.<br>
The Geofence should be either a Ring or Polygon type. <br>
For a Ring geofence **center and radius** are must.
For a Polygon geofence **vertices** of the polygon are must, there should be minimum of **3** points.<br>

GeoFence should be in the below mentioned format :
<font color="blue">_** Ring : ["geoFenceName", "Ring/RING/ring", [x,y(center)],Radius ] <br>
	Polygon : ["GeoFenceName", "polygon/Polygon/POLYGON",[[x,y],[x,y],[x,y],[x,y]....(vertices)]**_</font> <br>
<font color="red">_**Each data point is considered at second level i.e, 60 rows = 1 minute worth data**_</font><br>

**If the input latitude longitude values are contained in a CSV file, use the below code snippet to prepare the input JSON file**

In [43]:
import pandas as pd
import json

inputData = pd.read_csv('https://s3.amazonaws.com/sagemaker-sample-datasets/GeoFence/InputData/latLon.csv')
inputData.to_json('./sampleInput.json', orient='split')


fences = [["GeofenceP", "polygon", [[4, 9], [1, 2], [9, 7], [4, 6]] ],["GeofenceR", "ring", [0, 0], 4]]

with open('./sampleInput.json', 'r') as f:
    finalInputData = json.loads(f.read())
finalInputData['fences'] = fences
del(finalInputData['index'])

with open('./sampleInput.json', 'w') as f:
    f.write(json.dumps(finalInputData))

#inputData.head(10)
finalInputData

{'columns': ['latitude', 'longitude'],
 'data': [[200.0, 200.0],
  [33.5107978, -81.989483],
  [33.5107987, -81.989483],
  [33.5107995, -81.989484],
  [33.5107995, -81.989484],
  [33.5108002, -81.989485],
  [33.5108008, -81.989486],
  [33.5108008, -81.989487],
  [33.5108002, -81.989487],
  [33.5107997, -81.989487]],
 'fences': [['GeofenceP', 'polygon', [[4, 9], [1, 2], [9, 7], [4, 6]]],
  ['GeofenceR', 'ring', [0, 0], 4]]}

## 3. Using Endpoint

### 3.1. Creating Model
To create a model, import boto3, sagemaker and get the image name of the model 

In [ ]:
import boto3
import sagemaker

image = '<Place Image Id>'
role = sagemaker.get_execution_role()
sm = boto3.client('sagemaker')
modelName = '<Input Model Name>'
createHeatIndexResponse = sm.create_model(ModelName = modelName, ExecutionRoleArn = role, PrimaryContainer = {'Image':image})

### 3.2. Creating Endpoint Config


In [ ]:
configName='<Input Configuration Name>'
instanceType = '<Input Instance Type>'
createHeatIndexEndpointConfig = sm.create_endpoint_config(EndpointConfigName = configName, ProductionVariants = [{'InstanceType':instanceType, 'InitialInstanceCount':1, 'ModelName':modelName, 'VariantName':'xyz'}])

### 3.3. Creating Endpoint

In [ ]:
endpointName = '<Input Endpoint Name>'
createHeatIndexEndpoint = sm.create_endpoint(EndpointName = endpointName, EndpointConfigName = configName)

### 3.4. Invoking Endpoint

In [ ]:
runtime = boto3.Session().client('runtime.sagemaker')

#Reading Input Data 
with open('<Input json file path including json file name>', 'r') as f:
    payload = json.loads(f.read())

response = runtime.invoke_endpoint(EndpointName = endpointName, ContentType = 'application/json', Body = payload)
result = response['Body'].read().decode()

with open('<Output json file path including json file name>', 'w') as f:
    f.write(json.dumps(result))

### 3.5. Deleting the Endpoint - Optional

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(endpointName)

## 4. Using Batch Transform Job

Refer the below link for how to use batch transform job for getting inferences from a model
[sagemaker batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html#ex1-batch-transform-console)

In [ ]:
import boto3
import sagemaker

inputLocation = '<S3 location for Input Data>'
outputLocation = '<S3 location for Output Data>'

jobName = '<Input Job Name>'
modelName = '<Input Model Name>'
instanceType = '<Input Instance Type>'

# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(base_transform_job_name = jobName, model_name = modelName, instance_count=1, instance_type = instanceType, output_path = outputLocation)

# To start a transform job:
transformer.transform(inputLocation, content_type='text/csv', split_type='None')

# Then wait until transform job is completed
transformer.wait() 


## 5. Model Output

In [52]:
sampleOutput = pd.read_csv('https://s3.amazonaws.com/sagemaker-sample-datasets/GeoFence/OutputData/sampleOutput.json')
sampleOutput

,"{""Duration outside all geofences (in minutes)"": 0.05","""Duration in GeoFenceR (in minutes)"": 0.0","""Duration in GeoFenceP (in minutes)"": 0.0","""Total Duration (in minutes)"": 0.05}"
